In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt

In [2]:
airbnb_file = "AB_NYC_2019.csv"

In [3]:
airbnb_df = pd.read_csv(airbnb_file)
airbnb_df = airbnb_df.rename(columns={"host_id": "host_number", "host_name": "host_title",
                                      "neighbourhood_group":"borough"})

In [4]:
airbnb_df.head()

,id,name,host_number,host_title,borough,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [5]:
# Create a filtered dataframe from specific columns
host_cols = ["host_number", "host_title"]
host= airbnb_df[host_cols].copy()
host= host.drop_duplicates()
host.set_index("host_number",inplace=True)

In [6]:
host.head()

,host_title
host_number,
2787,John
2845,Jennifer
4632,Elisabeth
4869,LisaRoxanne
7192,Laura


In [7]:
room_cols=["host_number", "room_type"]
room_type=airbnb_df[room_cols].copy()
room_type.drop_duplicates("room_type", inplace=True)


In [8]:
room_type=room_type.drop(columns=["host_number"])

In [9]:
room_type.index=range(1,4)
room_type=room_type.reset_index()
room_type=room_type.rename(columns={"index":"room_type_id"})
room_type

,room_type_id,room_type
0,1,Private room
1,2,Entire home/apt
2,3,Shared room


In [10]:
neighb_cols=["host_number","borough"]
neighb=airbnb_df[neighb_cols].copy()
neighb.drop_duplicates("borough",inplace=True)

In [11]:
neighb=neighb.drop(columns=["host_number"])

In [12]:
neighb.index=range(1,6)
neighb=neighb.reset_index()
neighbourhood_group_transformed=neighb.rename(columns={"index":"borough_id"})
neighbourhood_group_transformed


,borough_id,borough
0,1,Brooklyn
1,2,Manhattan
2,3,Queens
3,4,Staten Island
4,5,Bronx


In [13]:
neighb_join=pd.DataFrame.merge(airbnb_df,neighbourhood_group_transformed)

In [14]:
neighb_join.head(5)

,id,name,host_number,host_title,borough,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,borough_id
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,1
1,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,1
2,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,45,49,2017-10-05,0.40,1,0,1
3,5803,"Lovely Room 1, Garden, Best Area, Legal rental",9744,Laurie,Brooklyn,South Slope,40.66829,-73.98779,Private room,89,4,167,2019-06-24,1.34,3,314,1
4,6848,Only 2 stops to Manhattan studio,15991,Allen & Irina,Brooklyn,Williamsburg,40.70837,-73.95352,Entire home/apt,140,2,148,2019-06-29,1.20,1,46,1


In [15]:
neighbourhood_df =  neighb_join[["borough_id","neighbourhood"]]
neighbourhood_df = neighbourhood_df.dropna(how='any')
neighbourhood_df_unique = neighbourhood_df.drop_duplicates()
neighbourhood_df_unique.index=range(1,222)
neighbourhood_df_unique=neighbourhood_df_unique.reset_index()
neighbourhood=neighbourhood_df_unique.rename(columns={"index":"neighbourhood_id"})

neighbourhood.head(10)

,neighbourhood_id,borough_id,neighbourhood
0,1,1,Kensington
1,2,1,Clinton Hill
2,3,1,Bedford-Stuyvesant
3,4,1,South Slope
4,5,1,Williamsburg
5,6,1,Fort Greene
6,7,1,Crown Heights
7,8,1,Park Slope
8,9,1,Windsor Terrace
9,10,1,Greenpoint


In [16]:
superJoin=pd.DataFrame.merge(neighb_join,neighbourhood)
superJoin["last_review"]=pd.to_datetime(superJoin["last_review"])

In [17]:
superJoin

,id,name,host_number,host_title,borough,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,borough_id,neighbourhood_id
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,1,1
1,63546,Large and Cozy Private Bedroom,308652,Antonín,Brooklyn,Kensington,40.64106,-73.97426,Private room,39,1,45,2019-05-16,0.46,2,365,1,1
2,125594,SUPER BIG AND COZY PRIVATE BEDROOM,308652,Antonín,Brooklyn,Kensington,40.64302,-73.97255,Private room,39,1,82,2019-05-19,0.94,2,365,1,1
3,267652,Private clean pleasant spacious room.,164675,Janice,Brooklyn,Kensington,40.64277,-73.97296,Private room,60,2,20,2019-01-21,0.42,1,347,1,1
4,282341,Kensington/Ditmas Park pied-a-terre,1471384,Dan,Brooklyn,Kensington,40.64330,-73.97386,Entire home/apt,90,30,3,2017-07-30,0.05,1,286,1,1
5,317905,Come and go as you please in BKLN!,1631733,Jane,Brooklyn,Kensington,40.64354,-73.97777,Entire home/apt,89,3,62,2019-01-02,0.71,1,189,1,1
6,452068,"Spacious 4 bedroom house, New York",2246071,Tamara,Brooklyn,Kensington,40.64205,-73.97173,Entire home/apt,200,1,3,2015-08-16,0.06,1,0,1,1
7,516791,Lovely Brooklyn Apt,696306,Bethany,Brooklyn,Kensington,40.64625,-73.97932,Entire home/apt,90,10,33,2019-05-22,0.39,1,269,1,1
8,656281,"QUIET, SPACIOUS, COMFORTABLE, & GREAT LOCATION",3180741,Manon,Brooklyn,Kensington,40.64573,-73.98013,Private room,72,1,48,2019-02-28,0.59,1,312,1,1
9,812467,"Vintage Chic Haven in Kensington, Brooklyn",574584,Jesse,Brooklyn,Kensington,40.63547,-73.97417,Private room,49,1,61,2019-06-23,0.95,1,100,1,1


In [18]:
listing=pd.DataFrame.merge(superJoin,room_type)
listing=listing.drop(columns=["calculated_host_listings_count","borough_id","host_title","borough","neighbourhood","room_type"])
listing

,id,name,host_number,latitude,longitude,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,availability_365,neighbourhood_id,room_type_id
0,2539,Clean & quiet apt home by the park,2787,40.64749,-73.97237,149,1,9,2018-10-19,0.21,365,1,1
1,63546,Large and Cozy Private Bedroom,308652,40.64106,-73.97426,39,1,45,2019-05-16,0.46,365,1,1
2,125594,SUPER BIG AND COZY PRIVATE BEDROOM,308652,40.64302,-73.97255,39,1,82,2019-05-19,0.94,365,1,1
3,267652,Private clean pleasant spacious room.,164675,40.64277,-73.97296,60,2,20,2019-01-21,0.42,347,1,1
4,656281,"QUIET, SPACIOUS, COMFORTABLE, & GREAT LOCATION",3180741,40.64573,-73.98013,72,1,48,2019-02-28,0.59,312,1,1
5,812467,"Vintage Chic Haven in Kensington, Brooklyn",574584,40.63547,-73.97417,49,1,61,2019-06-23,0.95,100,1,1
6,1157036,private spacious sunny bedroom,3696460,40.63381,-73.97140,50,30,10,2016-11-30,0.13,205,1,1
7,1239293,Cozy Retreat in Brooklyn Apartment,6755111,40.64120,-73.97860,75,2,8,2014-01-07,0.11,312,1,1
8,3645866,Welcome to Kensington,946672,40.63402,-73.97556,49,2,6,2016-09-05,0.10,0,1,1
9,3717749,Two Porches & Private Room,5573250,40.64130,-73.98233,70,1,2,2015-07-06,0.03,0,1,1


In [ ]:
pw = input("Password: ")
connection_string = "postgres:" + str(pw) + "@localhost:5432/airbnb_db"
engine = create_engine(f'postgresql://{connection_string}')

In [20]:
engine.table_names()

['borough', 'neighbourhood', 'listing', 'room_type', 'host']

In [21]:
host.to_sql(name='host', con=engine, if_exists='append', index=True)

In [26]:
neighbourhood_group_transformed.to_sql(name='borough', con=engine, if_exists='append', index=False)

In [29]:
neighbourhood.to_sql(name='neighbourhood', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "neighbourhood_pkey"
DETAIL:  Key (neighbourhood_id)=(1) already exists.

[SQL: INSERT INTO neighbourhood (neighbourhood_id, borough_id, neighbourhood) VALUES (%(neighbourhood_id)s, %(borough_id)s, %(neighbourhood)s)]
[parameters: ({'neighbourhood_id': 1, 'borough_id': 1, 'neighbourhood': 'Kensington'}, {'neighbourhood_id': 2, 'borough_id': 1, 'neighbourhood': 'Clinton Hill'}, {'neighbourhood_id': 3, 'borough_id': 1, 'neighbourhood': 'Bedford-Stuyvesant'}, {'neighbourhood_id': 4, 'borough_id': 1, 'neighbourhood': 'South Slope'}, {'neighbourhood_id': 5, 'borough_id': 1, 'neighbourhood': 'Williamsburg'}, {'neighbourhood_id': 6, 'borough_id': 1, 'neighbourhood': 'Fort Greene'}, {'neighbourhood_id': 7, 'borough_id': 1, 'neighbourhood': 'Crown Heights'}, {'neighbourhood_id': 8, 'borough_id': 1, 'neighbourhood': 'Park Slope'}  ... displaying 10 of 221 total bound parameter sets ...  {'neighbourho

In [30]:
room_type.to_sql(name='room_type', con=engine, if_exists='append', index=False)

In [31]:
listing.to_sql(name='listing', con=engine, if_exists='append', index=False)